<a href="https://colab.research.google.com/github/rkhushii/Codsoft-ML-task-1/blob/main/main%20file/spamham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np

In [ ]:
import os
import google.colab.drive as drive

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# Mount your Google Drive.
drive.mount('/content/drive',force_remount=True)

# List all of the files in your Drive.
files = os.listdir('/content/drive/My Drive/archive')

# Create a path to the file you want to load.
path = os.path.join('/content/drive/My Drive/archive', 'spam.csv')

# Load the file into a DataFrame.
data = pd.read_csv(path,encoding='ISO-8859-1')

Mounted at /content/drive


In [ ]:
data.tail()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN
5571,ham,Rofl. Its true to its name,NaN,NaN,NaN


In [ ]:
training_data=data[0:int(0.8*len(data))]
testing_data=data[int(0.8*len(data)):len(data)-1]

In [ ]:
train_messages=training_data['v2'].tolist()
train_messages

In [ ]:
test_messages=testing_data['v2'].tolist()

In [ ]:
train_labels=training_data['v1'].replace({'ham':0,'spam':1},inplace=False).tolist()
test_labels=testing_data['v1'].replace({'ham':0,'spam':1},inplace=False).tolist()

In [ ]:
vocab_dict={}
for message in train_messages:
  for word in message.split():
    if word not in vocab_dict:
      vocab_dict[word]=1
    else:
      vocab_dict[word]+=1
vocab_dict

In [ ]:
VOCAB_SIZE = len(vocab_dict)
MAX_LENGTH = len(max(train_messages, key=lambda p: len(p.split())).split())
print(VOCAB_SIZE,MAX_LENGTH)

13627 171


In [ ]:
encoded_train_messages = [one_hot(d, VOCAB_SIZE) for d in train_messages]
padded_train_messages = pad_sequences(encoded_train_messages, maxlen=MAX_LENGTH, padding='post')
encoded_test_messages = [one_hot(d, VOCAB_SIZE) for d in test_messages]
padded_test_messages = pad_sequences(encoded_test_messages, maxlen=MAX_LENGTH, padding='post')

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=100,mask_zero=True),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1,activation='relu')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(np.array(padded_train_messages),np.array( train_labels), validation_data=(np.array(padded_test_messages), np.array(test_labels)), epochs=5, verbose=2)


Epoch 1/5
140/140 - 33s - loss: 0.6932 - accuracy: 0.8649 - val_loss: 0.6931 - val_accuracy: 0.8698 - 33s/epoch - 237ms/step
Epoch 2/5
140/140 - 23s - loss: 0.6931 - accuracy: 0.8649 - val_loss: 0.6927 - val_accuracy: 0.8698 - 23s/epoch - 167ms/step
Epoch 3/5
140/140 - 26s - loss: 0.6652 - accuracy: 0.9129 - val_loss: 0.6269 - val_accuracy: 0.9758 - 26s/epoch - 184ms/step
Epoch 4/5
140/140 - 24s - loss: 0.6174 - accuracy: 0.9847 - val_loss: 0.6222 - val_accuracy: 0.9794 - 24s/epoch - 168ms/step
Epoch 5/5
140/140 - 26s - loss: 0.6089 - accuracy: 0.9919 - val_loss: 0.6313 - val_accuracy: 0.9820 - 26s/epoch - 182ms/step
